# Classification of the pulses
Function to classify each pulse in a signal from a file (folder file_to_classify). First, I need to use my Pearson function to calculate my correlation matrix between my 2 templates and the matrix of the pulses from a signal. Then I label every pulse good (value 1) or bad (value 0) and if I have 3 good pulses in a row my templates are updated. 

Inputs : threshold (th), learning rate (alpha), pulse matrix to classify, 2 templates

Outputs : 2 templates (updated or not), the list with baniry values as a result of the classification, 2 correlation matrices

In [1]:
import numpy as np
%run pearson.ipynb 

def classification(th,alpha,pulse_norm_art,master_temp_1, master_temp_2):
    rho1art = pearson(pulse_norm_art,master_temp_1) #correlation matrix between the matrix of pulses from a signal to classify and my 1st template
    rho2art = pearson(pulse_norm_art,master_temp_2) #correlation matrix between the matrix of pulses from a signal to classify and my 2nd template
    #output: rho has the same nbr of rows as first argument (=pulse_norm_art) of the function and same nbr of columns as the scnd argument (=template)
    # rho1art = np.ones((len(pulse_norm_art[0]), 1)) #values correlation matrix = 1 #just to try if the update was working  
    # rho2art = np.ones((len(pulse_norm_art[0]), 1))
    label=[]
    for i in range(len(rho1art)):
        rho1art=abs(rho1art) #values of the correlation matrix between -1 and 1 but to compare with the threshold has to be absolute values 
        rho2art=abs(rho2art)
        SQI=max(rho1art[i],rho2art[i]) #Signal Quality Index (SQI) will be compared with the threshold, SQI = max between 2 values of the 2 correlation matrices for the same row
        if SQI<th: #if the value of rho matrix is below the threshold the pulse is bad add 0 to a list 
            label.append(0) #0=bad, #1=good
        else: #the value of rho matrix is greater than the threshold pulse is good add 1 to a list
            label.append(1) #0=bad, #1=good
    #print(label)
        if i>=2 and i<len(rho1art):
            k=i-1
            if label[k]==1: #if the current pulse is good 
                if label[k-1]==0 and label[k+1]==0: #and if the the pulse before and after the previous pulse are bad
                    label[k]=0 #this pulse is not good #value 0
                else: 
                    master_temp_1=alpha*master_temp_1+(1-alpha)*pulse_norm_art[:,k] #update the templates 
                    master_temp_2=alpha*master_temp_2+(1-alpha)*pulse_norm_art[:,k] #equation: new template = alpha * current template + (1-alpha) * current pulse 
                    #alpha = learning rate 
                    rho1art = pearson(pulse_norm_art,master_temp_1) #because I have new templates, recalculation of the correlation matrix 
                    rho2art = pearson(pulse_norm_art,master_temp_2)       
    return master_temp_1, master_temp_2, label, rho1art, rho2art